In [45]:
!pip install gradio

In [48]:
import gradio as gr
from PIL import Image, ImageOps
from keras.models import load_model  # TensorFlow is required for Keras to work
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("/content/keras_model.h5", compile=False)

# Load the labels
class_names = open("/content/labels.txt", "r").readlines()

# Function to load and classify the image
def classify_image(image):
    # Convert the Gradio image to a PIL Image
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)

    # Convert image to numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    # Predict using the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    confidence_score = prediction[0][index]

    # Determine if the image is a dog or cat and create a result message
    confidence_threshold = 0.8  # Adjust this value as needed
    if confidence_score > confidence_threshold:
      result_text = f"This is a {class_names[index].strip()} with {confidence_score:.2f} confidence."
    else:
      result_text = "Unknown class"


    return result_text, result_text  # Return result text twice, once for the output and once for the flag

# Create and launch the Gradio interface
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),  # Ensure input is PIL Image
    outputs=[gr.Textbox(label="Output"), gr.Textbox(label="Flag")],  # Output to both the main output and the flag
    title="Image Classifier",
    description="Upload an image to classify it as a Cat or Dog."
)

interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b7ea97c5d67e6e120a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
